In [32]:
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi

DATASET_DIR = Path.cwd() / "datasets" / "dataturks"
DATASET_PATH = DATASET_DIR / "Entity Recognition in Resumes.json"

if DATASET_DIR.exists():
    print("Dataset already loaded")
else:
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files('dataturks/resume-entities-for-ner', path=DATASET_DIR, quiet=False, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/dataturks/resume-entities-for-ner


100%|██████████| 323k/323k [00:00<00:00, 156MB/s]

In [ ]:
import json
import pandas as pd
from collections import defaultdict

def pop_annot(raw_line):
    in_line = defaultdict(list, **raw_line)
    if 'annotation' in in_line:
        labels = in_line['annotation']
        for c_lab in labels:
            if len(c_lab['label'])>0:
                in_line[c_lab['label'][0]] += c_lab['points']
    return in_line

with open(DATASET_PATH, 'r') as f:
    resumes_data = [json.loads(f_line) for f_line in f.readlines()]
    resumes_df = pd.DataFrame([pop_annot(line) for line in resumes_data])
resumes_df['length'] = resumes_df['content'].map(len)
resumes_df.head()

,content,annotation,extras,Skills,College Name,Graduation Year,Designation,Companies worked at,Email Address,Location,Name,Degree,Years of Experience,UNKNOWN,length
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",None,"[{'start': 1295, 'end': 1621, 'text': ' • Prog...","[{'start': 939, 'end': 956, 'text': 'Kendriya ...","[{'start': 856, 'end': 860, 'text': '2017 '}]","[{'start': 727, 'end': 769, 'text': 'B.E in In...","[{'start': 407, 'end': 415, 'text': 'Accenture...","[{'start': 95, 'end': 145, 'text': 'Indeed: in...","[{'start': 60, 'end': 68, 'text': 'Bengaluru'}]","[{'start': 0, 'end': 11, 'text': 'Abhishek Jha'}]",NaN,NaN,NaN,1622
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",None,"[{'start': 743, 'end': 1140, 'text': 'Database...","[{'start': 675, 'end': 702, 'text': 'Shivaji U...","[{'start': 729, 'end': 732, 'text': '2016'}, {...",NaN,NaN,"[{'start': 1155, 'end': 1198, 'text': 'indeed....","[{'start': 62, 'end': 67, 'text': 'Sangli'}]","[{'start': 0, 'end': 13, 'text': 'Afreen Jamad...","[{'start': 631, 'end': 672, 'text': 'Bachelor ...",NaN,NaN,1240
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",None,"[{'start': 3749, 'end': 3756, 'text': 'Teradat...","[{'start': 3421, 'end': 3457, 'text': 'Anurag ...",NaN,"[{'start': 1620, 'end': 1642, 'text': 'Senior ...","[{'start': 1645, 'end': 1659, 'text': 'Infosys...","[{'start': 65, 'end': 116, 'text': 'indeed.com...","[{'start': 1664, 'end': 1672, 'text': 'Hyderab...","[{'start': 0, 'end': 20, 'text': 'Akhil Yadav ...","[{'start': 3381, 'end': 3419, 'text': 'Electri...",NaN,NaN,4264
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",None,"[{'start': 8098, 'end': 8383, 'text': '❖ Opera...","[{'start': 7955, 'end': 7991, 'text': 'Indira ...","[{'start': 7994, 'end': 7997, 'text': '2012'}]","[{'start': 1472, 'end': 1509, 'text': 'Operati...","[{'start': 1512, 'end': 1517, 'text': 'UNISYS'...","[{'start': 2391, 'end': 2432, 'text': 'indeed....","[{'start': 5016, 'end': 5024, 'text': 'Bengalu...","[{'start': 0, 'end': 11, 'text': 'Alok Khandai'}]",NaN,NaN,NaN,8384
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",None,"[{'start': 973, 'end': 1702, 'text': 'EARCH EN...","[{'start': 922, 'end': 938, 'text': 'Mumbai Un...",NaN,"[{'start': 795, 'end': 802, 'text': 'lecturer'...","[{'start': 24, 'end': 40, 'text': ' oracle tut...","[{'start': 85, 'end': 129, 'text': 'indeed.com...","[{'start': 943, 'end': 948, 'text': 'Mumbai'},...","[{'start': 0, 'end': 12, 'text': 'Ananya Chava...","[{'start': 2010, 'end': 2012, 'text': 'MCA'}, ...",NaN,NaN,2933
